In [1]:
import os
import pandas as pd
import numpy as np
import neurokit2 as nk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPooling1D
from tqdm import tqdm

# Constants
ecg_folder = "../../../../Datasets/12-lead electrocardiogram database/ECGDataDenoised"
diagnostics_file = "../../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"

# Label mapping
rhythm_mapping = {
    'AFIB': 'AFIB',
    'AF': 'AFIB',
    'SVT': 'GSVT',
    'AT': 'GSVT',
    'SAAWR': 'GSVT',
    'ST': 'GSVT',
    'AVNRT': 'GSVT',
    'AVRT': 'GSVT',
    'SB': 'SB',
    'SR': 'SR',
    'SA': 'SR'
}

# Load diagnostics data
diagnostics_df = pd.read_excel(diagnostics_file)
diagnostics_df['Rhythm'] = diagnostics_df['Rhythm'].map(rhythm_mapping)


# Helper function to preprocess and extract features using NeuroKit2
def preprocess_ecg(ecg_signal, sampling_rate=500):
    try:
        # Process the ECG signal
        signals, info = nk.ecg_process(ecg_signal, sampling_rate=sampling_rate)

        # Extract NeuroKit2-calculated features
        features = signals.mean().to_dict()
        return features
    except Exception as e:
        print(f"Error during preprocessing: {e}")
        return None


# Process files and extract features
data = []
labels = []
errors = []

for idx, row in tqdm(diagnostics_df.iterrows(), total=len(diagnostics_df)):
    ecg_file = os.path.join(ecg_folder, row["FileName"] + ".csv")
    if os.path.exists(ecg_file):
        try:
            lead_ii = pd.read_csv(ecg_file, header=0).iloc[:, 1].values  # Extract Lead II data
            features = preprocess_ecg(lead_ii)
            if features is not None:
                data.append(features)
                labels.append(row["Rhythm"])
            else:
                errors.append((row["FileName"], "Feature extraction failed"))
        except Exception as e:
            errors.append((row["FileName"], str(e)))
    else:
        errors.append((row["FileName"], "File not found"))

# Log errors
if errors:
    print("Files with errors:")
    for file, error in errors:
        print(f"{file}: {error}")

# Convert to DataFrame
data = pd.DataFrame(data)
labels = pd.Series(labels)

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)

2024-12-11 19:02:33.158342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 19:02:33.221303: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 19:02:33.239146: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 19:02:33.351394: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-11 19:02:34.143475: W tensorflow/compiler/tf2

Error during preprocessing: cannot convert float NaN to integer


 10%|▉         | 1064/10646 [01:13<11:03, 14.44it/s]/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/neurokit2/signal/signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
 10%|█         | 1067/10646 [01:13<10:06, 15.79it/s]

Error during preprocessing: cannot convert float NaN to integer


 12%|█▏        | 1309/10646 [01:30<10:39, 14.59it/s]/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/neurokit2/signal/signal_fixpeaks.py:307: RuntimeWarning: divide by zero encountered in divide
  mrrs /= th2
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/neurokit2/signal/signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
 14%|█▍        | 1473/10646 [01:41<10:35, 14.43it/s]/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/neuroki

...

Error during preprocessing: integer division or modulo by zero


 99%|█████████▉| 10585/10646 [12:19<00:05, 11.63it/s]/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/neurokit2/signal/signal_fixpeaks.py:307: RuntimeWarning: divide by zero encountered in divide
  mrrs /= th2
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/neurokit2/signal/signal_fixpeaks.py:307: RuntimeWarning: invalid value encountered in divide
  mrrs /= th2
100%|█████████▉| 10605/10646 [12:21<00:03, 11.25it/s]/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/neuro

Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo 

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/neurokit2/signal/signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in sc

Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo by zero
Error during preprocessing: integer division or modulo by zero
Files with errors:
MUSE_20180114_124812_70000: Feature extraction failed
MUSE_20180115_122527_35000: Feature extraction failed
MUSE_20180112_170005_52000: Feature extraction failed
MUSE_20180712_152821_61000: Feature extraction failed
MUSE_20180116_121910_44000: Feature extraction failed
MUSE_20180113_124215_52000: Feature extraction failed
MUSE_20180114_070744_27000: Feature extraction failed
MUSE_20180113_131440_08000: Feature extraction failed
MUSE_20180114_121833_58000: Feature extraction failed
MUSE_20180712_154131_30000: Feature extraction failed
MUSE_20180114_121037_94000: Feature extraction failed
MUSE_20180113_121039_40000: Feature extraction failed
MUSE_20180712_151825_22000: Feature extraction failed
MUSE_20180114_124930_50000:

In [2]:
# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
print("Decision Tree Classification Report:")
print(classification_report(y_test, y_pred_dt, target_names=label_encoder.classes_, digits=5))

# MLP Model
mlp_model = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(len(label_encoder.classes_), activation='softmax')
])
mlp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mlp_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)
y_pred_mlp = np.argmax(mlp_model.predict(X_test), axis=1)
print("MLP Classification Report:")
print(classification_report(y_test, y_pred_mlp, target_names=label_encoder.classes_, digits=5))

# CNN Model
cnn_model = Sequential([
    tf.keras.layers.Reshape((X_train.shape[1], 1), input_shape=(X_train.shape[1],)),
    Conv1D(32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(label_encoder.classes_), activation='softmax')
])
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)
y_pred_cnn = np.argmax(cnn_model.predict(X_test), axis=1)
print("CNN Classification Report:")
print(classification_report(y_test, y_pred_cnn, target_names=label_encoder.classes_, digits=5))

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1733922897.393652   48518 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


Decision Tree Classification Report:
              precision    recall  f1-score   support

        AFIB    0.70686   0.74398   0.72495       457
        GSVT    0.82367   0.79418   0.80866       447
          SB    0.94466   0.93238   0.93848       769
          SR    0.87810   0.88209   0.88009       441

    accuracy                        0.85194      2114
   macro avg    0.83832   0.83816   0.83804      2114
weighted avg    0.85379   0.85194   0.85269      2114



I0000 00:00:1733922897.538936   48518 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733922897.541346   48518 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733922897.545546   48518 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733922897.547735   48518 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Epoch 1/50


I0000 00:00:1733922898.225784   57274 service.cc:146] XLA service 0x7779dc004070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733922898.225816   57274 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-12-11 19:14:58.259128: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-11 19:14:58.368715: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


155/265 ━━━━━━━━━━━━━━━━━━━━ 0s 984us/step - accuracy: 0.5033 - loss: 1.1281

I0000 00:00:1733922899.427898   57274 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


265/265 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5683 - loss: 1.0139
Epoch 2/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7948 - loss: 0.5741
Epoch 3/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step - accuracy: 0.8316 - loss: 0.4781
Epoch 4/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 890us/step - accuracy: 0.8433 - loss: 0.4515
Epoch 5/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 984us/step - accuracy: 0.8479 - loss: 0.4351
Epoch 6/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 614us/step - accuracy: 0.8615 - loss: 0.3952
Epoch 7/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 582us/step - accuracy: 0.8671 - loss: 0.3858
Epoch 8/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - accuracy: 0.8691 - loss: 0.3732
Epoch 9/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 534us/step - accuracy: 0.8672 - loss: 0.3876
Epoch 10/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 482us/step - accuracy: 0.8707 - loss: 0.3628
Epoch 11/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 474us/step - accuracy: 0.8689 - loss: 0.3683
Epoch 12/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 5

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/reshaping/reshape.py:39: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


265/265 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.6678 - loss: 0.8444
Epoch 2/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 999us/step - accuracy: 0.8291 - loss: 0.4980
Epoch 3/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 657us/step - accuracy: 0.8547 - loss: 0.4240
Epoch 4/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step - accuracy: 0.8647 - loss: 0.4025
Epoch 5/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step - accuracy: 0.8650 - loss: 0.3930
Epoch 6/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 928us/step - accuracy: 0.8735 - loss: 0.3724
Epoch 7/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step - accuracy: 0.8722 - loss: 0.3715
Epoch 8/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 834us/step - accuracy: 0.8781 - loss: 0.3478
Epoch 9/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8812 - loss: 0.3376
Epoch 10/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8801 - loss: 0.3472
Epoch 11/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8837 - loss: 0.3250
Epoch 12/50
265/265 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/s